# ETL to get the text data from the playlist

This notebook shows the process of building the corpus of transcripts from the YouTube playlist.

**Extract**: Pull data (transcripts) from each video.  
**Transform**:  
**Load**: Load data into our database where it will be retrieved from.  

In [1]:
from models import etl
import json

First we load the video information. This includes the video IDs and titles.

In [2]:
with open('data/single_video.json') as f:
    video_info = json.load(f)

Then we must extract the transcripts using the YouTube Transcript API. This is done over all of the videos.  
This produces a list of video segments with timestamps.  
Next, we format the transcript by adding metadata so that the segments are easily identified for retreival later.  
Since the original segments are small, they are batched with overlap to preserve semantic meaning.

In [3]:
videos = []
for video in video_info:
    video_id = video["id"]
    video_title = video["title"]
    transcript = etl.get_video_transcript(video_id)
    print(f"Transcript for video {video_id} fetched.")
    if transcript:
        formatted_transcript = etl.format_transcript(transcript, video_id, video_title, batch_size=5, overlap=2)
        
        videos.extend(formatted_transcript)

get_video_transcript took 0.84 seconds.
Transcript for video 5sLYAQS9sWQ fetched.


The last step is to load the data into a database. We will use a Chromadb database.  
The embedding function is the ____ model from HuggingFace.

In [4]:
# Initialize the database
from utils.embedding_utils import MyEmbeddingFunction
import chromadb

embed_text = MyEmbeddingFunction()

db_path = "data/single_video.db"
client = chromadb.PersistentClient(path=db_path)

client.create_collection(
    name="huberman_videos",
    embedding_function=embed_text,
    metadata={"hnsw:space": "cosine"}
)

print(f"Database created at {db_path}")

Database created at data/single_video.db


In [5]:
# Add the data to the database
client = chromadb.PersistentClient(path=db_path)
    
collection = client.get_collection("huberman_videos")

documents = [segment['text'] for segment in videos]
metadata = [segment['metadata'] for segment in videos]
ids = [segment['metadata']['segment_id'] for segment in videos]

collection.add(
    documents=documents,
    metadatas=metadata,
    ids=ids
)

print(f"Data loaded to database at {db_path}.")

Data loaded to database at data/single_video.db.


Here is some of the data:

In [8]:
import pandas as pd

client = chromadb.PersistentClient('data/single_video.db')
collection= client.get_collection('huberman_videos')

num_segments = collection.count()
sample_data = collection.peek()

transcript_df = pd.DataFrame(sample_data)

print(f"Number of segments: {num_segments}")
transcript_df

Number of segments: 26


,ids,embeddings,metadatas,documents,uris,data
0,5sLYAQS9sWQ__0,"[-0.11489544063806534, -0.03262839838862419, -...","{'segment_id': '5sLYAQS9sWQ__0', 'source': 'ht...","GPT, or Generative Pre-trained Transformer, is...",None,None
1,5sLYAQS9sWQ__12,"[0.094169981777668, -0.10430295020341873, 0.02...","{'segment_id': '5sLYAQS9sWQ__12', 'source': 'h...",Now foundation models are pre-trained on large...,None,None
2,5sLYAQS9sWQ__15,"[0.042587604373693466, -0.061460819095373154, ...","{'segment_id': '5sLYAQS9sWQ__15', 'source': 'h...","I'm talking about things like code. Now, large...",None,None
3,5sLYAQS9sWQ__18,"[-0.0245895367115736, -0.058405470103025436, -...","{'segment_id': '5sLYAQS9sWQ__18', 'source': 'h...",these models can be tens of gigabytes in size ...,None,None
4,5sLYAQS9sWQ__21,"[0.05348338559269905, -0.016104578971862793, -...","{'segment_id': '5sLYAQS9sWQ__21', 'source': 'h...","So to put that into perspective, a text file t...",None,None
5,5sLYAQS9sWQ__24,"[0.07004527002573013, -0.08996045589447021, -0...","{'segment_id': '5sLYAQS9sWQ__24', 'source': 'h...",A lot of words just in one Gb. And how many gi...,None,None
6,5sLYAQS9sWQ__27,"[0.0283487681299448, -0.11020224541425705, -0....","{'segment_id': '5sLYAQS9sWQ__27', 'source': 'h...","Yeah, that's truly a lot of text. And LLMs are...",None,None
7,5sLYAQS9sWQ__3,"[-0.0700172707438469, -0.061202701181173325, -...","{'segment_id': '5sLYAQS9sWQ__3', 'source': 'ht...",And I've been using GPT in its various forms f...,None,None
8,5sLYAQS9sWQ__30,"[-0.04904637485742569, -0.1277533322572708, -0...","{'segment_id': '5sLYAQS9sWQ__30', 'source': 'h...","and the more parameters a model has, the more ...",None,None
9,5sLYAQS9sWQ__33,"[0.03286760300397873, -0.041724931448698044, 0...","{'segment_id': '5sLYAQS9sWQ__33', 'source': 'h...","All right, so how do they work? Well, we can t...",None,None
